## Разработка Telegram-бота с использованием aiogram 3.x

### 1. Установка и настройка

#### Установка библиотеки

```bash
pip install aiogram
```

#### Создание бота в Telegram

1. Перейдите к боту `@BotFather` в Telegram.
2. Используйте команду `/newbot` для создания нового бота.
3. Сохраните токен вашего бота.

### 2. Основные компоненты aiogram

#### **Bot**

Класс `Bot` предоставляет доступ к Telegram API.

```python
from aiogram import Bot

bot = Bot(token="YOUR_BOT_TOKEN")
```

#### **Dispatcher**

`Dispatcher` управляет обработчиками событий.

```python
from aiogram import Dispatcher

dp = Dispatcher()
```

#### **Handlers (Обработчики)**

Обработчики обрабатывают события, такие как команды, текстовые сообщения или нажатия кнопок.

```python
from aiogram.types import Message
from aiogram import Router

router = Router()

@router.message()
async def handle_message(message: Message):
    await message.reply(f"Вы отправили: {message.text}")
```

#### **Middleware**

Middleware позволяет выполнять действия до и после обработки события.

```python
from aiogram import BaseMiddleware
from aiogram.types import Message

class LoggingMiddleware(BaseMiddleware):
    async def __call__(self, handler, event: Message, data: dict):
        print(f"Получено сообщение: {event.text}")
        return await handler(event, data)
```

### 3. Создание простого бота

#### Файл `bot.py`

```python
import asyncio
from aiogram import Bot, Dispatcher
from aiogram.types import Message
from aiogram.filters import Command

# Создаем экземпляры бота и диспетчера
bot = Bot(token="YOUR_BOT_TOKEN")
dp = Dispatcher()

# Обработчик команды /start
@dp.message(Command("start"))
async def cmd_start(message: Message):
    await message.reply("Добро пожаловать! Я ваш бот.")

# Обработчик команды /help
@dp.message(Command("help"))
async def cmd_help(message: Message):
    await message.reply("Я могу ответить на команды /start и /help.")

# Основная функция запуска бота
async def main():
    print("Бот запущен!")
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())
```


### 4. Обработка команд, текста и кнопок

#### Обработка текста

```python
@dp.message()
async def echo_message(message: Message):
    await message.reply(f"Вы сказали: {message.text}")
```

#### Инлайн-клавиатуры

```python
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton

keyboard = InlineKeyboardMarkup(
    inline_keyboard=[
        [InlineKeyboardButton(text="Кнопка 1", callback_data="btn1")],
        [InlineKeyboardButton(text="Кнопка 2", callback_data="btn2")],
    ]
)

@dp.message(Command("keyboard"))
async def show_keyboard(message: Message):
    await message.reply("Выберите опцию:", reply_markup=keyboard)

@dp.callback_query()
async def handle_callback(callback_query):
    if callback_query.data == "btn1":
        await callback_query.message.reply("Вы нажали Кнопка 1")
    elif callback_query.data == "btn2":
        await callback_query.message.reply("Вы нажали Кнопка 2")
```

### 5. Работа с состояниями (FSM)

**FSM** (Finite State Machine) позволяет управлять сложными сценариями.

#### Определение состояний

```python
from aiogram.fsm.state import State, StatesGroup

class Form(StatesGroup):
    name = State()
    age = State()
```

#### Использование состояний

```python
from aiogram.fsm.context import FSMContext

@dp.message(Command("form"))
async def start_form(message: Message, state: FSMContext):
    await message.reply("Как вас зовут?")
    await state.set_state(Form.name)

@dp.message(Form.name)
async def process_name(message: Message, state: FSMContext):
    await state.update_data(name=message.text)
    await message.reply("Сколько вам лет?")
    await state.set_state(Form.age)

@dp.message(Form.age)
async def process_age(message: Message, state: FSMContext):
    data = await state.get_data()
    name = data.get("name")
    age = message.text
    await message.reply(f"Привет, {name}! Тебе {age} лет.")
    await state.clear()
```

### 6. Расширенные возможности

#### Работа с API

```python
import aiohttp

@dp.message(Command("joke"))
async def get_joke(message: Message):
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.chucknorris.io/jokes/random") as response:
            joke = await response.json()
            await message.reply(joke["value"])
```

### 7. Деплой бота

#### Использование Docker

Создайте файл `Dockerfile`:

```dockerfile
FROM python:3.10

WORKDIR /app

COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "bot.py"]
```

Соберите и запустите образ:

```bash
docker build -t my_telegram_bot .
docker run -d --name my_telegram_bot my_telegram_bot
```


## Конвертер валют


curl -X GET "https://api.apilayer.com/exchangerates_data/latest?base=USD" \
-H "apikey: API_KEY"

curl --request GET 'https://api.apilayer.com/exchangerates_data/latest?base=USD&symbols=EUR,GBP' --header 'apikey: API_KEY'  


Создадим Telegram-бота, который позволяет пользователям конвертировать валюты в реальном времени с использованием открытого API для курсов валют. Бот будет:

1. Обрабатывать команды `/start` и `/help`.
2. Спрашивать у пользователя валюту для конвертации.
3. Отправлять курсы валют, полученные из API.
4. Поддерживать клавиатуры для удобного выбора валют.

Для получения данных о курсах валют будем использовать бесплатный API, например, [ExchangeRate-API](https://apilayer.com/marketplace/exchangerates_data-api). Зарегистрируйтесь и получите **API-ключ**.

#### 1. Файл `config.py`

```python
import os

from dotenv import load_dotenv

# Суперподробное логирование для отладки
import logging
logging.basicConfig(level=logging.DEBUG)

aiohttp_logger = logging.getLogger("aiohttp")
aiohttp_logger.setLevel(logging.DEBUG)

# Загрузка переменных из .env файла
load_dotenv()

API_TOKEN = os.getenv("BOT_TOKEN")
CURRENCY_API_KEY = os.getenv("API_TOKEN")
CURRENCY_API_URL = "https://api.apilayer.com/exchangerates_data/latest"

if not API_TOKEN or not CURRENCY_API_KEY:
    raise NameError

```

#### 2. Файл `utils.py`

```python
import asyncio
import aiohttp
from config import CURRENCY_API_KEY, CURRENCY_API_URL


async def get_exchange_rate(base_currency: str, target_currency: str) -> float | None:
    headers = {"apikey": CURRENCY_API_KEY}
    params = {"base": base_currency.upper()}

    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(CURRENCY_API_URL, headers=headers, params=params, timeout=10) as response:
                if response.status == 200:
                    data = await response.json()
                    rates = data.get("rates", {})
                    return rates.get(target_currency.upper())
                else:
                    print(f"Ошибка API: {response.status}, {await response.text()}")
        except aiohttp.ClientError as e:
            print(f"Ошибка клиента API: {e}")
        except asyncio.TimeoutError:
            print("Ошибка: Таймаут при запросе к API")
    return None

```

#### 3. Основной файл `bot.py`

```python
import asyncio
from aiogram import Bot, Dispatcher
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, Message
from aiogram.filters import Command
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from config import API_TOKEN
from utils import get_exchange_rate

bot = Bot(token=API_TOKEN)
dp = Dispatcher()


class CurrencyConversion(StatesGroup):
    base_currency = State()
    target_currency = State()


currency_keyboard = ReplyKeyboardMarkup(
    keyboard=[
        [KeyboardButton(text="USD"), KeyboardButton(text="EUR"), KeyboardButton(text="RUB")],
        [KeyboardButton(text="USD to RUB"), KeyboardButton(text="EUR to RUB")],
    ],
    resize_keyboard=True,
)


@dp.message(Command("start"))
async def start_command(message: Message):
    await message.answer(
        "Привет! Я бот, который проверяет текущий курс валют. Поддерживается два режима:\n"
        "1. В сценарии /convert : введите базовую валюту, потом валюту, в которую вы хотите сконвертировать;\n"
        "2. Введите запрос в формате 'USD to RUB.'\n"
        "Популярные валюты и конвертации представлены в виде кнопок.\n",
        reply_markup=currency_keyboard,
    )


@dp.message(Command("help"))
async def help_command(message: Message):
    await message.answer(
        "Привет! Я бот, который проверяет текущий курс валют. Поддерживается два режима:\n"
        "1. В сценарии /convert : введите базовую валюту, потом валюту, в которую вы хотите сконвертировать;\n"
        "2. Введите запрос в формате 'USD to RUB.'\n"
        "Популярные валюты и конвертации представлены в виде кнопок.\n"
        "/start - Начать работу\n"
        "/help - Помощь"
    )


@dp.message(Command("convert"))
async def start_conversion(message: Message, state: FSMContext):
    """Начало конверсии валют через FSM."""
    await state.set_state(CurrencyConversion.base_currency)
    await message.answer("Выберите базовую валюту:", reply_markup=currency_keyboard)


@dp.message(CurrencyConversion.base_currency)
async def select_base_currency(message: Message, state: FSMContext):
    """Выбор базовой валюты."""
    await state.update_data(base_currency=message.text)
    await state.set_state(CurrencyConversion.target_currency)
    await message.answer("Теперь выберите валюту для конвертации:", reply_markup=currency_keyboard)


@dp.message(CurrencyConversion.target_currency)
async def select_target_currency(message: Message, state: FSMContext):
    """Выбор валюты для конверсии."""
    data = await state.get_data()
    base_currency = data["base_currency"]
    target_currency = message.text

    rate = await get_exchange_rate(base_currency, target_currency)  # Асинхронный запрос
    if rate is None:
        await message.answer("Ошибка при получении курса.")
    else:
        await message.answer(f"Курс {base_currency} к {target_currency}: {rate:.5f}")

    await state.clear()


@dp.message()
async def convert_currency(message: Message):
    """Обработка текстовых запросов в формате 'USD to RUB'."""
    try:
        parts = message.text.split(" to ")
        if len(parts) != 2:
            await message.answer("Введите запрос в формате 'USD to RUB'.")
            return

        base_currency, target_currency = parts
        rate = await get_exchange_rate(base_currency, target_currency)

        if rate is None:
            await message.answer("Не удалось получить курс. Убедитесь, что валюты указаны корректно.")
            return

        await message.answer(f"Курс {base_currency} к {target_currency}: {rate:.5f}")
    except Exception as e:
        await message.answer("Произошла ошибка при обработке запроса.")


async def main():
    try:
        await dp.start_polling(bot)
    finally:
        await bot.session.close()

if __name__ == "__main__":
    asyncio.run(main())

```


In [1]:
import requests

API_KEY = ""
API_URL = "https://api.apilayer.com/exchangerates_data/latest"

headers = {"apikey": API_KEY}
params = {"base": "USD"}

response = requests.get(API_URL, headers=headers, params=params)

if response.status_code == 200:
    print(response.json())
else:
    print(f"Ошибка: {response.status_code}, {response.text}")


{'success': True, 'timestamp': 1735434064, 'base': 'USD', 'date': '2024-12-29', 'rates': {'AED': 3.673042, 'AFN': 70.483863, 'ALL': 94.154318, 'AMD': 400.326092, 'ANG': 1.804345, 'AOA': 912.000367, 'ARS': 1028.503429, 'AUD': 1.608752, 'AWG': 1.8, 'AZN': 1.70397, 'BAM': 1.875797, 'BBD': 2.021484, 'BDT': 119.666235, 'BGN': 1.875849, 'BHD': 0.377116, 'BIF': 2960.629166, 'BMD': 1, 'BND': 1.360284, 'BOB': 6.917949, 'BRL': 6.19575, 'BSD': 1.001199, 'BTC': 1.0531219e-05, 'BTN': 85.655781, 'BWP': 13.925095, 'BYN': 3.276459, 'BYR': 19600, 'BZD': 2.011125, 'CAD': 1.44175, 'CDF': 2870.000362, 'CHF': 0.902164, 'CLF': 0.035991, 'CLP': 992.480698, 'CNY': 7.298804, 'CNH': 7.300404, 'COP': 4398.407903, 'CRC': 507.936508, 'CUC': 1, 'CUP': 26.5, 'CVE': 105.754568, 'CZK': 24.175604, 'DJF': 178.286098, 'DKK': 7.152504, 'DOP': 60.892917, 'DZD': 135.548842, 'EGP': 50.85791, 'ERN': 15, 'ETB': 127.756678, 'EUR': 0.958904, 'FJD': 2.322404, 'FKP': 0.791982, 'GBP': 0.795418, 'GEL': 2.810391, 'GGP': 0.791982, 'GH